### Final Project: Wetland Restoration Suitability Map

This project aims to use cost surface analysis to identify high quality areas for wetland restoration projects. High quality wetland restoration areas are defined as places with hydrologic connection and proximity to existing wetland areas.

____

### Part 1: API Requests and File Extraction/Transformation

#### 0. Import Libraries and Establish Common Terms

In [8]:
# Import Libraries
import io
import os
import sys
import arcpy
import laspy
from zipfile import ZipFile
import zipfile
import pandas as pd
import requests
import ftplib
from bs4 import BeautifulSoup as bs
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureCollection, FeatureSet
from arcpy import env
from arcpy.sa import *

In [2]:
# Common Terms for all layers

# Boundary Extent (not watershed boundary)
# Top	4,887,308.789800	m
# Bottom	4,868,689.051000	m
# Left	379,749.855800	m
# Right	399,383.588100	m

#### 1a. NWI API Extraction

In [23]:
# Extract NWI Data

# Define the URL and the output path for the ZIP file (MN State Wetlands NWI)
url = r"https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/water_nat_wetlands_inv_2009_2014/fgdb_water_nat_wetlands_inv_2009_2014.zip"
zip_MN_NWI_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\ZippedFiles\NWI_MN.zip"

# Create the folder if it doesn't exist
os.makedirs(os.path.dirname(zip_MN_NWI_path), exist_ok=True)

# Download the file using requests.get(), stream=True for chunked downloading
response = requests.get(url, stream=True)

# Get the file size from headers (optional, for progress indication)
file_size = int(response.headers.get('content-length', 0))
print(f"Downloading NWI MN ZIP file ({file_size / 1024 / 1024:.2f} MB)...")

# Stream the download and show a progress bar
with open(zip_MN_NWI_path, 'wb') as f:
    with tqdm(total=file_size, unit='B', unit_scale=True, desc="Downloading") as pbar:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:  # Filter out keep-alive new chunks
                f.write(chunk)
                pbar.update(len(chunk))

# Open the ZIP file and print its contents
with zipfile.ZipFile(zip_MN_NWI_path, 'r') as zip_ref:
    # Print the contents of the ZIP file
    zip_ref.printdir()

    # Create a directory for extraction if it doesn't exist
    extraction_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Extracted\NWI_MN"
    os.makedirs(extraction_path, exist_ok=True)

    # Extract all files to the specified directory
    zip_ref.extractall(extraction_path)

# Show the files you just extracted
print("Files in the extracted directory:")
print(os.listdir(extraction_path))

Downloading: 100%|██████████| 1.61G/1.61G [00:56<00:00, 28.7MB/s]﻿


File Name                                             Modified             Size
NWI Cowardin Classification.lyr                2019-06-07 13:45:14        22016
NWI USFWS Circular 39 Classification.lyr       2019-06-07 13:45:46        20992
NWI Simplified HGM Classification.lyr          2019-06-07 13:45:22        18432
NWI Simplified Plant Community Classification.lyr 2019-06-07 13:45:36        28160
metadata/metadata.html                         2024-03-28 16:42:04        41011
metadata/metadata.xml                          2024-03-28 16:41:56        38116
water_nat_wetlands_inv_2009-2014.gdb/a00000038.gdbtable 2019-06-20 17:04:42          594
water_nat_wetlands_inv_2009-2014.gdb/a00000037.gdbtable.cdf 2019-06-20 17:04:42         2975
water_nat_wetlands_inv_2009-2014.gdb/a00000039.gdbtable.cdf 2019-06-20 17:04:42       310343
water_nat_wetlands_inv_2009-2014.gdb/a00000004.gdbtable 2019-06-20 17:36:24        32773
water_nat_wetlands_inv_2009-2014.gdb/a00000006.gdbindexes 2015-04-01 21:2

#### 1b. Identify Wetland Buffer Area

#### 2a. NCLD API Extraction

In [24]:
# Extract NCLD Land Cover Data

# Define the URL and the output path for the ZIP file
url = r"https://s3-us-west-2.amazonaws.com/mrlc/nlcd_2021_land_cover_l48_20230630.zip"
zip_ncld_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\ZippedFiles\NCLD_2021.zip"

# Create the folder if it doesn't exist
os.makedirs(os.path.dirname(zip_ncld_path), exist_ok=True)

# Download the file using requests.get(), stream=True for chunked downloading
response = requests.get(url, stream=True)

# Get the file size from headers (optional, for progress indication)
file_size = int(response.headers.get('content-length', 0))
print(f"Downloading NLCD ZIP file ({file_size / 1024 / 1024:.2f} MB)...")

# Stream the download and show a progress bar
with open(zip_ncld_path, 'wb') as f:
    with tqdm(total=file_size, unit='B', unit_scale=True, desc="Downloading") as pbar:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:  # Filter out keep-alive new chunks
                f.write(chunk)
                pbar.update(len(chunk))

# Open the ZIP file and print its contents
with zipfile.ZipFile(zip_ncld_path, 'r') as zip_ref:
    # Print the contents of the ZIP file
    zip_ref.printdir()

    # Create a directory for extraction if it doesn't exist
    extraction_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Extracted\NLCD_2021"
    os.makedirs(extraction_path, exist_ok=True)

    # Extract all files to the specified directory
    zip_ref.extractall(extraction_path)

# Show the files you just extracted
print("Files in the extracted directory:")
print(os.listdir(extraction_path))

Downloading: 100%|██████████| 1.96G/1.96G [01:06<00:00, 29.3MB/s]﻿


File Name                                             Modified             Size
nlcd_2021_land_cover_l48_20230630.ige          2023-07-10 13:24:40  26350989969
nlcd_2021_land_cover_l48_20230630.img          2023-07-10 15:35:52    435940485
nlcd_2021_land_cover_l48_20230630.xml          2023-07-11 14:56:44        48467
Files in the extracted directory:
['nlcd_2021_land_cover_l48_20230630.ige', 'nlcd_2021_land_cover_l48_20230630.img', 'nlcd_2021_land_cover_l48_20230630.xml']


#### 2b. Identify Developed Lands and Agricultural/Vegetation Lands

In [41]:
# Clip the NCLD Raster to Bounding Box and Reproject as NAD 83 UTM 15N

with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAVD_1988",VDATUM["North_American_Vertical_Datum_1988"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]', snapRaster=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped\slope_dem", extent='379749.8558 4868689.051 399383.5881 4887308.7898 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAVD_1988",VDATUM["North_American_Vertical_Datum_1988"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]'):
    arcpy.management.Clip(
        in_raster=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Extracted\NLCD_2021\nlcd_2021_land_cover_l48_20230630.img",
        rectangle="379749.8558 4868689.051 399383.5881 4887308.7898",
        out_raster=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped\NCLD_clipped",
        in_template_dataset="restorable_wetlands.tif",
        nodata_value="255",
        clipping_geometry="NONE",
        maintain_clipping_extent="MAINTAIN_EXTENT"
    )

In [52]:
# Path to your raster
raster_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped\NCLD_clipped"

# Get raster properties
desc = arcpy.Describe(raster_path)
cell_size = desc.meanCellWidth  # or desc.meanCellHeight
print(f"Original Cell Size: {cell_size}")

Original Cell Size: 29.975163816793987


In [45]:
# Reclassify the NCLD from 16 Categories to 5 Categories

with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAVD_1988",VDATUM["North_American_Vertical_Datum_1988"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]', snapRaster=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped\slope_dem", extent='379749.8558 4868689.051 399383.5881 4887308.7898 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAVD_1988",VDATUM["North_American_Vertical_Datum_1988"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]'):
        arcpy.ddd.Reclassify(
            in_raster="NCLD_clipped",
            reclass_field="VALUE",
            remap="11 5;21 4;22 4;23 4;24 4;31 4;41 2;42 2;43 2;52 2;71 2;81 3;82 3;90 1;95 1",
            out_raster=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped\NCLD_reclass",
            missing_values="DATA"
        )

In [46]:
# Convert the NCLD Raster to Polygon for Use in Buffering

with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAVD_1988",VDATUM["North_American_Vertical_Datum_1988"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]'):
    arcpy.conversion.RasterToPolygon(
        in_raster="NCLD_reclass",
        out_polygon_features=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped\NCLD_polygon.shp",
        simplify="SIMPLIFY",
        raster_field="VALUE",
        create_multipart_features="SINGLE_OUTER_PART",
        max_vertices_per_feature=None
    )

In [45]:
# Create a 100 m buffer around all impervious (developed) features

with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAVD_1988",VDATUM["North_American_Vertical_Datum_1988"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]'):
    arcpy.analysis.PairwiseBuffer(
        in_features="NCLD_polygon",
        out_feature_class=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Extracted\NLCD_2021\NCLD_Buffer.shp",
        buffer_distance_or_field="100 Meters",
        dissolve_option="ALL",
        dissolve_field=None,
        method="GEODESIC",
        max_deviation="0 Meters"
    )

In [53]:
# Convert the buffer feature layer to raster
arcpy.conversion.PolygonToRaster(
    in_features=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Extracted\NLCD_2021\NCLD_Buffer.shp",
    value_field="FID",
    out_rasterdataset=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped\buffer_raster",
    cell_assignment="MAXIMUM_AREA",
    priority_field="NONE",
    cellsize=29.975163816793987,
    build_rat="BUILD"
)

<Result 'C:\\Users\\ethan\\Desktop\\ARLT_MGIS\\GIS5571\\FinalProject\\Data\\Clipped\\buffer_raster'>

In [54]:
# Path to your raster
raster_path = "buffer_raster"

# Get raster properties
desc = arcpy.Describe(raster_path)
cell_size = desc.meanCellWidth  # or desc.meanCellHeight
print(f"Original Cell Size: {cell_size}")

Original Cell Size: 29.975163816794037


#### 3a. DEM API Extraction

In [25]:
# Extract DEM Data for Blue Earth, Watonwan, and Brown Counties

# Define the base URL and the counties to check
DOMAIN = "https://resources.gisdata.mn.gov"
BASE_URL = f"{DOMAIN}/pub/data/elevation/lidar/county"
counties = ["blueearth", "brown", "watonwan"]

# Directory where the files will be saved
save_folder = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\ZippedFiles\MN_County_DEM"
os.makedirs(save_folder, exist_ok=True)  # Create the folder if it doesn't exist

# Loop through counties to download and extract the file
for county in counties:
    file_url = f"{BASE_URL}/{county}/elevation_data.gdb.zip"
    file_path = os.path.join(save_folder, f"{county}_elevation_data.gdb.zip")
    
    # Check if the file exists on the server
    response = requests.head(file_url)
    if response.status_code == 200:
        print(f"Found file for {county} county: Downloading...")

        # Download the file with progress bar
        response = requests.get(file_url, stream=True)
        file_size = int(response.headers.get('content-length', 0))
        
        with open(file_path, 'wb') as f, tqdm(total=file_size, unit='B', unit_scale=True, desc=f"Downloading {county}") as pbar:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))

        print(f"Downloaded {file_path}")

        # Extract the ZIP file
        extraction_path = os.path.join(save_folder, f"{county}_extracted")
        os.makedirs(extraction_path, exist_ok=True)
        
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(extraction_path)
        
        print(f"Extracted files to {extraction_path}")

    else:
        print(f"No file found for {county} county.")

Found file for blueearth county: Downloading...


Downloaded C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\ZippedFiles\MN_County_DEM\blueearth_elevation_data.gdb.zip
Extracted files to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\ZippedFiles\MN_County_DEM\blueearth_extracted
Found file for brown county: Downloading...


Downloaded C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\ZippedFiles\MN_County_DEM\brown_elevation_data.gdb.zip
Extracted files to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\ZippedFiles\MN_County_DEM\brown_extracted
Found file for watonwan county: Downloading...


Downloaded C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\ZippedFiles\MN_County_DEM\watonwan_elevation_data.gdb.zip
Extracted files to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\ZippedFiles\MN_County_DEM\watonwan_extracted


In [30]:
# Add County DEM Rasters to Mosaic Database and clip to Bounding Box Extent

with arcpy.EnvManager(extent='379749.8558 4868689.051 399383.5881 4887308.7898 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAVD_1988",VDATUM["North_American_Vertical_Datum_1988"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]'):
    arcpy.management.MosaicToNewRaster(
        input_rasters=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Extracted\blueearth_extracted\elevation_data.gdb\dem_3m_m;C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Extracted\brown_extracted\elevation_data.gdb\dem_3m_m;C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Extracted\watonwan_extracted\elevation_data.gdb\dem_3m_m",
        output_location=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped",
        raster_dataset_name_with_extension="Mosaic_DEM",
        coordinate_system_for_the_raster='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAVD_1988",VDATUM["North_American_Vertical_Datum_1988"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]',
        pixel_type="8_BIT_UNSIGNED",
        cellsize=None,
        number_of_bands=1,
        mosaic_method="LAST",
        mosaic_colormap_mode="FIRST"
    )

#### 3b. Create Slope and Total Water Index (TWI) Rasters

In [31]:
# Derive Slope from the DEM raster mosaic layer

with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped"):
    out_raster = arcpy.sa.Slope(
        in_raster="Mosaic_DEM",
        output_measurement="DEGREE",
        z_factor=1,
        method="PLANAR",
        z_unit="METER",
        analysis_target_device="GPU_THEN_CPU"
    )
    out_raster.save(r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped\slope_DEM")

In [34]:
# Derive TWI from the DEM raster mosaic layer -- NOT WORKING AS CODE WITH TOOLBOX INSTALLED

# Load the Arc Hydro Pro toolbox
arcpy.ImportToolbox(r"C:\Users\ethan\Documents\ArcGIS\Projects\Arc1_FinalProject\Arc1_FinalProject.atbx")

arcpy.archydropro.calculatetopographicwetnessindexsurfaceparameters(
    Input_Smoothed_DEM="Mosaic_DEM",
    Hydrocondition_Method="Continuous Flow",
    Slope_Neighborhood_Distance=10,
    Output_TWI_Raster=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\predictor_variables\Mosaic_DEM_twi_SLP_an_10mMax_CF.tif",
    Save_Intermediate_Rasters=False,
    Output_TWI_Slope_Raster=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\secondary_outputs\Mosaic_DEM_SLP_an_10mMax.tif",
    Output_TWI_FAC_Raster=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\secondary_outputs\Mosaic_DEM_FAC_CF.tif",
    Input_TWI_Slope=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped\slope_DEM",
    Input_TWI_FAC_Raster=None
)

AttributeError: module 'arcpy' has no attribute 'archydropro'

#### 3c. Create Layers for Specific Slope and TWI Values

Use the DEM derived slope and TWI rasters to create new cost layers: 1) slope <= 7* and 2) TWI >=7.

In [35]:
# Create a Slope Cost Layer (degrees <= 5).

# Reclassify the Categories by Slope (0-5* = 1, <=10 = 2, <=Max = 3)
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAVD_1988",VDATUM["North_American_Vertical_Datum_1988"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]'):
    arcpy.ddd.Reclassify(
        in_raster="out_raster",
        reclass_field="VALUE",
        remap="0 5 1;5 10 2;10 66.788101 3",
        out_raster=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped\Slope_Reclass",
        missing_values="DATA"
    )

In [36]:
# Create a TWI Cost Layer (TWI >= 7)

# Reclassify the Categories by TWI (TWI 0-5 = 1, TWI <=10 = 2, <=20 = 3, >20 = 4)

with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAVD_1988",VDATUM["North_American_Vertical_Datum_1988"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]'):
    arcpy.ddd.Reclassify(
        in_raster="Mosaic_DEM_twi_SLP_an_10mMax_CF.tif",
        reclass_field="VALUE",
        remap="-2.738983 5 1;5 10 2;10 20 3;20 24.795912 4",
        out_raster=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped\TWI_reclass",
        missing_values="DATA"
    )

ExecuteError: ERROR 010237: Error in building STA while build VAT.
ERROR 010067: Error in executing grid expression.
Failed to execute (Reclassify).


#### 4a. Soils Data API Extraction

In [26]:
# Extract Soils Data for Bounding Box
   
# Define the URL and the output path for the ZIP file
url = r"https://websoilsurvey.sc.egov.usda.gov/DSD/Download/AOI/vnuor1iisg3slwbdgugqbp0w/wss_aoi_2024-11-01_09-54-19.zip"
zip_soils_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\ZippedFiles\MN_Soils.zip"

# Create the folder if it doesn't exist
os.makedirs(os.path.dirname(zip_ncld_path), exist_ok=True)

# Download the file using requests.get(), stream=True for chunked downloading
response = requests.get(url, stream=True)

# Get the file size from headers (optional, for progress indication)
file_size = int(response.headers.get('content-length', 0))
print(f"Downloading MN Soils Data file ({file_size / 1024 / 1024:.2f} MB)...")

# Stream the download and show a progress bar
with open(zip_soils_path, 'wb') as f:
    with tqdm(total=file_size, unit='B', unit_scale=True, desc="Downloading") as pbar:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:  # Filter out keep-alive new chunks
                f.write(chunk)
                pbar.update(len(chunk))

# Open the ZIP file and print its contents
with zipfile.ZipFile(zip_soils_path, 'r') as zip_ref:
    # Print the contents of the ZIP file
    zip_ref.printdir()

    # Create a directory for extraction if it doesn't exist
    extraction_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Extracted\MN_Soils"
    os.makedirs(extraction_path, exist_ok=True)

    # Extract all files to the specified directory
    zip_ref.extractall(extraction_path)

# Show the files you just extracted
print("Files in the extracted directory:")
print(os.listdir(extraction_path))

Downloading: 100%|██████████| 21.0M/21.0M [00:22<00:00, 919kB/s] ﻿


File Name                                             Modified             Size
wss_aoi_2024-11-01_09-54-19/readme.txt         2024-11-01 09:54:54        20733
wss_aoi_2024-11-01_09-54-19/soildb_MN_2003.mdb 2012-09-13 11:24:42     12406784
wss_aoi_2024-11-01_09-54-19/soil_metadata_mn013.txt 2024-11-01 09:54:54        58967
wss_aoi_2024-11-01_09-54-19/soil_metadata_mn013.xml 2024-11-01 09:54:54        56665
wss_aoi_2024-11-01_09-54-19/soil_metadata_mn015.txt 2024-11-01 09:54:54        60929
wss_aoi_2024-11-01_09-54-19/soil_metadata_mn015.xml 2024-11-01 09:54:54        59006
wss_aoi_2024-11-01_09-54-19/soil_metadata_mn165.txt 2024-11-01 09:54:54        57245
wss_aoi_2024-11-01_09-54-19/soil_metadata_mn165.xml 2024-11-01 09:54:54        55131
wss_aoi_2024-11-01_09-54-19/thematic/          2024-11-01 09:54:54            0
wss_aoi_2024-11-01_09-54-19/spatial/aoi_a_aoi.dbf 2024-11-01 09:54:18           92
wss_aoi_2024-11-01_09-54-19/spatial/aoi_a_aoi.prj 2024-11-01 09:54:18          145
wss_

#### 4b. Identify Hydric Soils

In [27]:
#arc_api_link = f"https://landscape11.arcgis.com/arcgis/rest/services/USA_Soils_Hydric_Class/ImageServer/query?where=1%3D1&outFields=*&f=pjson"

# Currently Issues accessing the REST API for layer, downloaded manually

IndentationError: unexpected indent (<string>, line 4)

In [29]:
# Reproject the Soils Raster into NAD 83 UTM 15N

arcpy.management.ProjectRaster(
    in_raster="hydric_soils.tif",
    out_raster=r"C:\Users\ethan\Documents\ArcGIS\Projects\Arc1_FinalProject\Arc1_FinalProject.gdb\hydric_soils_Project",
    out_coor_system='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAVD_1988",VDATUM["North_American_Vertical_Datum_1988"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]',
    resampling_type="NEAREST",
    cell_size="29.9999961896375 30",
    geographic_transform="WGS_1984_(ITRF00)_To_NAD_1983",
    Registration_Point=None,
    in_coor_system='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]',
    vertical="NO_VERTICAL"
)

<Result 'C:\\Users\\ethan\\Documents\\ArcGIS\\Projects\\Arc1_FinalProject\\Arc1_FinalProject.gdb\\hydric_soils_Project'>

In [32]:
# Clip the Reprojected Soils Raster to the Boundary Extent

with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAVD_1988",VDATUM["North_American_Vertical_Datum_1988"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]'):
    arcpy.management.Clip(
        in_raster="hydric_soils_Project",
        rectangle="379749.142119867 4868686.82685342 399384.142119867 4887310.82685342",
        out_raster=r"C:\Users\ethan\Documents\ArcGIS\Projects\Arc1_FinalProject\Arc1_FinalProject.gdb\hydric_soils_Clip",
        in_template_dataset="Mosaic_DEM_twi_SLP_an_10mMax_CF.tif",
        nodata_value="255",
        clipping_geometry="NONE",
        maintain_clipping_extent="NO_MAINTAIN_EXTENT"
    )

#### Create a Soils Cost Layer

Create a cost layer where hydric soils over 50 percent are the highest value.

In [33]:
# Reclassify Soils Data

arcpy.ddd.Reclassify(
    in_raster="hydric_soils_Clip",
    reclass_field="ClassName",
    remap="'Not Hydric' 1;'Partially Hydric (1 - 25%)' 2;'Partially Hydric (26 - 50%)' 3;'Mostly Hydric (76 - 95%)' 4;'All Hydric' 5",
    out_raster=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped\soils_reclass",
    missing_values="DATA"
)

<Result 'C:\\Users\\ethan\\Desktop\\ARLT_MGIS\\GIS5571\\FinalProject\\Data\\Clipped\\soils_reclass'>

____

### Part 2: Develop a Cost Surface Layer and Identify High Priority Areas

#### 5a. Create a Cost Surface Layer

In [55]:
# Create an Equal Cost Surface using NCLD 1 and 3 = 10, TWI > 5 = 10, Slope < 10 = 10, Soil Hydric > 50% = 10

with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Outputs"):
    out_raster = arcpy.sa.WeightedOverlay(
        in_weighted_overlay_table=r"('C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped\ncld_reclass' 40 'VALUE' (1 10; 2 5; 3 10; 4 1; 5 1; NODATA NODATA); 'C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped\reclass_twi' 20 'VALUE' (1 1; 2 10; 3 10; 4 10; NODATA NODATA); 'C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped\slope_reclass' 20 'VALUE' (1 10; 2 5; 3 1; NODATA NODATA); 'C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Data\Clipped\soils_reclass' 20 'VALUE' (1 1; 2 3; 3 5; 4 10; 5 10; NODATA NODATA));1 10 1"
    )
    out_raster.save(r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\FinalProject\Outputs\EqWgt_NoBuff")

#### 5b. Evaluate Cost Surface for High Value Wetlands

Determine wetland areas that are 2 cells or larger.

In [56]:


# Identify areas with 2 cells the same touching
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAVD_1988",VDATUM["North_American_Vertical_Datum_1988"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]', scratchWorkspace=r"C:\Users\ethan\Documents\ArcGIS\Projects\Arc1_FinalProject\Arc1_FinalProject.gdb"):
    out_raster = arcpy.sa.FocalStatistics(
        in_raster="out_raster",
        neighborhood="Rectangle 3 3 CELL",
        statistics_type="MEAN",
        ignore_nodata="DATA",
        percentile_value=90
    )
    out_raster.save(r"C:\Users\ethan\Documents\ArcGIS\Projects\Arc1_FinalProject\Arc1_FinalProject.gdb\FocalSt_EqWg1")

In [57]:
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAVD_1988",VDATUM["North_American_Vertical_Datum_1988"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]', scratchWorkspace=r"C:\Users\ethan\Documents\ArcGIS\Projects\Arc1_FinalProject\Arc1_FinalProject.gdb"):
    out_raster = arcpy.sa.FocalStatistics(
        in_raster="out_raster",
        neighborhood="Circle 10 CELL",
        statistics_type="MEAN",
        ignore_nodata="DATA",
        percentile_value=90
    )
    out_raster.save(r"C:\Users\ethan\Documents\ArcGIS\Projects\Arc1_FinalProject\Arc1_FinalProject.gdb\FocalSt_EqWg2")

#### 5c. Identify Land Use by Parcel and Protection Status

_____

### Part 3: Analyze Wetland Site Priorities

#### 6a. Randomized Weighting

#### 6b. Compare to Non-Randomized Weighting (Equal Weighting)

#### 6c. Autocorrelation Results